In [1]:
import tensorflow as tf
import os
from PIL import Image
import numpy as np

In [2]:
def count_samples():
    path = os.path.join(orig_picture + classes_1[CLASSES_NUM])
    print('path  ' + path)
    #path = 'F:/Desktop/TF2/picture/centrum/'    #获取当前路径
 
    num_dirs = 0 #路径下文件夹数量
    num_files = 0 #路径下文件数量(包括文件夹)
    num_files_rec = 0 #路径下文件数量,包括子文件夹里的文件数量，不包括空文件夹

    for root,dirs,files in os.walk(path):    #遍历统计
            for each in files:
                    num_files_rec += 1
    print(num_files_rec)
    return num_files_rec

In [3]:
#识别分别
CLASSES_NUM = 2 #改这个就行了 0开始

classes_1 = ['cube', 'brightMilk', 'budweiser', 'centrum', 'iceTea']
classes = {classes_1[CLASSES_NUM]}#'brightMilk', 'budweiser', 'centrum', 'cube', 
#dir_name = np.array(['1', '2', '3', '4', '5'])
dir_name = ['1', '2', '3', '4', '5']

#原始图片地址
orig_picture = 'F:/Desktop/TF2/picture/'
#生成图片地址
gen_picture = os.path.join( 'F:/Desktop/TF2/processed/' + dir_name[CLASSES_NUM] + '/')
print('gen_picture  ' + gen_picture)

#TFRecord输出地址
tfrecord_path = os.path.join('F:/Desktop/TF2/tfrecord/' + classes_1[CLASSES_NUM] + '.tfrecords')
print('tfrecord_path  ' + tfrecord_path)
#样本总数
num_samples = count_samples() #600*5

gen_picture  F:/Desktop/TF2/processed/3/
tfrecord_path  F:/Desktop/TF2/tfrecord/budweiser.tfrecords
path  F:/Desktop/TF2/picture/budweiser
388


In [4]:
#制作TFRecords数据  
def create_record():  
    writer = tf.python_io.TFRecordWriter(tfrecord_path)  
    for index, name in enumerate(classes):  
        class_path = orig_picture +"/"+ name+"/"  
        for img_name in os.listdir(class_path):  
            img_path = class_path + img_name  
            img = Image.open(img_path)  
            img = img.resize((64, 64)).convert("RGB")    #设置需要转换的图片大小
            
            # img=img.convert("L")       #图片灰度化 L = R * 299/1000 + G * 587/1000 + B * 114/1000
            img_raw = img.tobytes()      #将图片转化为字符串，用decode_raw解码
            
            #print(index, np.array(img_raw))#读入到一个numpy数组中，看一下数据格式
            #print(index,img_raw)  
            example = tf.train.Example(  
               features=tf.train.Features(feature={  
                    "label": tf.train.Feature(int64_list=tf.train.Int64List(value=[index])),  #value来自for
                    'img_raw': tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw]))  
               }))  
            writer.write(example.SerializeToString())  
    writer.close()  

In [5]:
#读取 解码
def read_and_decode(filename):  
    # 创建文件队列,不限读取的数量  
    filename_queue = tf.train.string_input_producer([filename])  
    # create a reader from file queue  
    reader = tf.TFRecordReader()  
    # reader从文件队列中读入一个序列化的样本  
    _, serialized_example = reader.read(filename_queue)  #Return a tuple of Tensors (key, value).
    # get feature from serialized example  
    # 解析符号化的样本  
    features = tf.parse_single_example(  
        serialized_example,  
        features={  
            #FixedLenFeature 返回定长的张量
            'label': tf.FixedLenFeature([], tf.int64),   #编号
            'img_raw': tf.FixedLenFeature([], tf.string) #名称 
        })  
    label = features['label']  
    img = features['img_raw']  
    img = tf.decode_raw(img, tf.uint8)  #将原来编码为字符串类型的变量重新变回来
    img = tf.reshape(img, [64, 64, 3])  #64*64*3

    label = tf.cast(label, tf.int32)  #数据类型的转变，不会改变原始数据的值还有形状
    return img, label  

In [6]:
def main():
    #制作TFR
    #create_record()
    #解析样本 获取batch
    batch = read_and_decode(tfrecord_path)#batch = [img, label]
    #初始化
    init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer())
    
    with tf.Session() as sess:
        sess.run(init_op)
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord = coord)
        
        for i in range(num_samples):
            example, lab = sess.run(batch)
            img = Image.fromarray(example, 'RGB')
            img.save(gen_picture + '/' + str(i) + '_' + 'G'  + '_' +'samples' + str(lab) + '.jpg')
            print('dir0 = ', gen_picture + '/' + str(i) + '_' + 'G'  + '_' +'samples' + str(lab) + '.jpg')
            #解开后启用图像增强，速度慢
#             for j in range(5):
#                 if j == 0:#上下翻转
#                     img = tf.image.flip_up_down(img)
#                 elif j == 1:#左右翻转
#                     img = tf.image.flip_left_right(img)
#                 elif j == 2:#旋转90°
#                     img = tf.image.transpose_image(img)
#                 elif j == 3:#旋转-90°
#                     img = tf.image.rot90(img, k=4)
#                 elif j == 4:#加入noise
#                     img = img + tf.cast( tf.random_uniform(shape=[64, 64, 3],
#                                minval=8,
#                                maxval=12), tf.uint8)
#                 encoded_image = tf.image.encode_jpeg(img)
#                 file_full_write = os.path.join(gen_picture + '/' + str(i) + '_' + str(j)  + '_' +'samples' + str(lab) + '.jpg')
#                 with tf.gfile.GFile(file_full_write, "wb") as f:
#                     f.write(encoded_image.eval())
#                 print('dir1 = ', gen_picture + '/' + str(i) + '_' + str(j)  + '_' +'samples' + str(lab) + '.jpg')
            print('example = ')#,print(example)
            print('-----------------------------------------------------------------')
            print('lab='),print(lab)
            print('=================================================================')
        coord.request_stop()
        coord.join(threads)
        sess.close()

In [7]:
main()

dir0 =  F:/Desktop/TF2/processed/3//0_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//1_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//2_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//3_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//4_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//5_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//6_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//7_G_s

dir0 =  F:/Desktop/TF2/processed/3//55_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//56_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//57_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//58_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//59_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//60_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//61_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3

dir0 =  F:/Desktop/TF2/processed/3//111_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//112_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//113_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//114_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//115_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//116_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//117_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/proc

example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//162_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//163_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//164_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//165_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//166_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//167_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//168_G_samples0.jpg
example = 
----------------

dir0 =  F:/Desktop/TF2/processed/3//206_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//207_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//208_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//209_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//210_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//211_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//212_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/proc

dir0 =  F:/Desktop/TF2/processed/3//251_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//252_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//253_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//254_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//255_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//256_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//257_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/proc

dir0 =  F:/Desktop/TF2/processed/3//310_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//311_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//312_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//313_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//314_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//315_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//316_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/proc

dir0 =  F:/Desktop/TF2/processed/3//373_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//374_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//375_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//376_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//377_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//378_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/processed/3//379_G_samples0.jpg
example = 
-----------------------------------------------------------------
lab=
0
dir0 =  F:/Desktop/TF2/proc